# Data Collection & Preparation

## 1.Import python packages

In [1]:
import pandas as pd
import numpy as np
import os
file_path='C:/Business_Data_Analysis/'
# file_path=os.environ['USERPROFILE']+'/Business_Data_Analysis/'

## 2.Establish connection to database

In [2]:
import pyodbc
import urllib
import sqlalchemy

'''sources databases'''
params_p = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost\SQLEXPRESS;"
                                 "DATABASE=DataWarehouse1;"
                                 "UID=sa;"
                                 "PWD=user1")

engine_p = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_p))


params_s = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost\SQLEXPRESS;"
                                 "DATABASE=DataWarehouse2;"
                                 "UID=sa;"
                                 "PWD=user1")

engine_s = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_s))



'''destination databases'''

params_datahub = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost\SQLEXPRESS;"
                                 "DATABASE=datahub;"
                                 "UID=sa;"
                                 "PWD=user1")

engine_datahub = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_datahub))

## 3.back up your data

In [3]:
'''get historical data from source database'''
df_sales_p=pd.read_sql_table('sales_data_primary',engine_p)

from datetime import datetime
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S")
print('Current Timestamp : ', timestampStr)

df_sales_p.to_csv(file_path+'data/sales_data_primary_backup_'+timestampStr+'.csv',index=False)

Current Timestamp :  24-Feb-2022-20-14-19


## 4.ETL data to datahub

In [4]:
df_sales_s=pd.read_sql('sales_data_secondary',engine_s)

df_sales_s.to_sql("sales_data_secondary",engine_datahub,if_exists='replace',index=False)

## a. Append new data to historical data

### i. Option 1: using python

In [5]:
from datetime import datetime
# dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y')

# df = pd.read_csv(infile, parse_dates=['onboarded_month','user_login_month','auth_month'], date_parser=dateparse)


'''get new data'''
# df_sales_p_new=pd.read_csv(file_path+'data/sales_data_class_new.csv',parse_dates=['onboarded_month','user_login_month','auth_month'], date_parser=dateparse,index_col=False)
df_sales_p_new=pd.read_csv(file_path+'data/sales_data_class_new.csv',parse_dates=['onboarded_month','user_login_month','auth_month'],index_col=False)
fmt='%m/%d/%Y'
'''get historical data from source database'''
df_sales_p=pd.read_sql_table('sales_data_primary',engine_p)


'''append new data to historical data'''
df_sales_p=pd.concat([df_sales_p,df_sales_p_new])

'''drop duplicate'''
df_sales_p.drop_duplicates(keep='first', inplace=True)

df_sales_p.shape

(10011, 11)

### ii. Check data

In [6]:
df_sales_p_new.shape,df_sales_p.shape

((17, 11), (10011, 11))

In [7]:
df_sales_s.tail()

,user_login_month,count_user_login,auth_month,registration_status,primary_act_holder_member_id,transaction_amount,purchase_amount,purchase_count,receive_amount,receive_count
3995,2021-01-01,35,NaT,activated,341,48.66,1354.30,59,585.10,97
3996,2021-01-01,1,2021-01-01,activated,722,49.52,123.85,88,1121.15,40
3997,2021-01-01,65,2021-01-01,activated,314,44.22,1538.85,22,1218.18,88
3998,2021-01-01,58,NaT,activated,442,63.95,1194.69,77,16.67,30
3999,2021-01-01,73,NaT,activated,253,67.81,1455.40,90,1176.96,72


### iii. Export updated historical data to database

In [8]:
df_sales_p.to_sql("sales_data_primary", engine_datahub,if_exists='replace',index=False)

### i. Option 2: using SQL

In [9]:
from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%m/%d/%Y')


'''get new data'''
df_sales_p_new=pd.read_csv(file_path+'data/sales_data_class_new.csv',parse_dates=['onboarded_month','user_login_month','auth_month'], index_col=False)


'''export new data into datahub'''
df_sales_p_new.to_sql("sql_sales_data_primary_new", engine_datahub,if_exists='replace',index=False) 

'''append new data to historical data'''

server = 'localhost\SQLEXPRESS' 
database = 'datahub' 
username = 'sa' 
password = 'user1' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

sql='''

DROP TABLE IF EXISTS #temp1

select * into #temp1
from (
select * from [dbo].[sql_sales_data_primary]
union 
select * from [dbo].[sql_sales_data_primary_new]) a

drop table if EXISTS [dbo].[sql_sales_data_primary]

select * into [dbo].[sql_sales_data_primary]
from #temp1

'''

cursor.execute(sql)
cnxn.commit()
cursor.close()

